In [1]:
from pyspark.sql import SparkSession

In [3]:
# 2.12 => scala version
# 3.1.2 => pyspark version
ss = SparkSession.builder\
                .appName('kafka-spark')\
                .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
                .getOrCreate()
ss

:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lab17/.ivy2/cache
The jars for the packages stored in: /home/lab17/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94633f1d-22d0-4342-a2ed-1d6aed8daf03;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.

In [4]:
kafka_stream = ss.readStream \
                .format('kafka')\
                .option('kafka.bootstrap.servers', 'localhost:9092')\
                .option('subscribe', 'kafka-test')\
                .load()

kafka_stream

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [5]:
kafka_data = kafka_stream.selectExpr('CAST(value AS STRING)')
kafka_data

DataFrame[value: string]

In [6]:
query = kafka_data.writeStream\
                .outputMode('append')\
                .format('console')\
                .start()
query

24/12/18 13:36:56 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-34e19ef4-8af3-4828-a144-27d02a47e3b6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+
|              value|
+-------------------+
|kafaka_pyspark|
+-------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
|  hmm|
+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|hmmmmmmmmmmmmmmmmmmm|
+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|tes...|
+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------------------+
|                           value|
+--------------------------------+
|긴 문자를 입력해보자 긴 문...|
+-----------------------

### 옵션 설정 테스트

In [ ]:

kafka_stream = ss.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='3 seconds') \
    .start()

print("Streaming is running...")
query.awaitTermination()


24/12/18 13:41:21 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-de6403aa-5a11-4451-b1fc-dfada52528a0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Streaming is running...
-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|               h|
|               hello|
|                test|
|                 wow|
|        this is good|
|             message|
|             한글도?|
|               wow !|
|          wo|
| i'm second producer|
|                 wow|
|             wow !!!|
|                 hey|
|                 hi~|
|heelo guy...|
|       hello guy|
|         hello guys~|
|                 hi~|
|           Message 0|
|           Message 1|
+--------------------+
only showing top 20 rows



### 데이터 처리 테스트

In [ ]:

kafka_stream = ss.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

# 모든 실행 중인 쿼리를 확인하고 중지
for query in spark.streams.active:
    if query.name == "kafka_messages":
        query.stop()
        
query = kafka_data.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("kafka_messages") \
    .start()

print("Streaming is running...")

import time
while True:
    spark.sql("SELECT * FROM kafka_messages").show()
    time.sleep(5)


In [9]:
ss.stop()